In [1]:
pip install mlxtend

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 8.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Import Libraries

In [25]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

### Read Data

In [4]:
retail_data = pd.read_excel("Online Retail.xlsx")

### Data Understanding

In [5]:
retail_data.shape

(541909, 8)

In [7]:
retail_data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
retail_data.describe(include="all")

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909.0,541909,540455,541909.000000,541909,541909.000000,406829.000000,541909
unique,25900.0,4070,4223,NaN,NaN,NaN,NaN,38
top,573585.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,NaN,United Kingdom
freq,1114.0,2313,2369,NaN,NaN,NaN,NaN,495478
mean,NaN,NaN,NaN,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570,NaN
min,NaN,NaN,NaN,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,2011-03-28 11:34:00,1.250000,13953.000000,NaN
50%,NaN,NaN,NaN,3.000000,2011-07-19 17:17:00,2.080000,15152.000000,NaN
75%,NaN,NaN,NaN,10.000000,2011-10-19 11:27:00,4.130000,16791.000000,NaN
max,NaN,NaN,NaN,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000,NaN


In [8]:
retail_data["Country"].nunique()

38

In [9]:
retail_data["Description"].nunique()

4223

In [10]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [11]:
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [23]:
retail_data.groupby(by="Country")["InvoiceNo"].count()

Country
Australia                 1259
Austria                    401
Bahrain                     19
Belgium                   2069
Brazil                      32
Canada                     151
Channel Islands            758
Cyprus                     622
Czech Republic              30
Denmark                    389
EIRE                      8196
European Community          61
Finland                    695
France                    8557
Germany                   9495
Greece                     146
Hong Kong                  288
Iceland                    182
Israel                     297
Italy                      803
Japan                      358
Lebanon                     45
Lithuania                   35
Malta                      127
Netherlands               2371
Norway                    1086
Poland                     341
Portugal                  1519
RSA                         58
Saudi Arabia                10
Singapore                  229
Spain                     2533


### Data Preparation/Cleaning

In [12]:
del retail_data['CustomerID']

In [13]:
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,United Kingdom


In [14]:
retail_data.dropna(axis=0,inplace=True)

In [15]:
retail_data.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
Country        0
dtype: int64

In [16]:
retail_data.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
Country                object
dtype: object

In [24]:
# United Kingdom, France, Germany

uk_data = retail_data[retail_data["Country"]=="United Kingdom"]
france_data = retail_data[retail_data["Country"]=="France"]
germany_data = retail_data[retail_data["Country"]=="Germany"]

In [36]:
# Pivoting UK Data
uk_pivot_data = pd.pivot_table(data=uk_data,values= "Quantity",index="InvoiceNo",columns="Description",fill_value=0)
uk_pivot_data = uk_pivot_data.applymap(lambda x:1 if x>0 else 0)

C:\Users\Chetan\AppData\Local\Temp\ipykernel_23240\3219876129.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  uk_pivot_data = uk_pivot_data.applymap(lambda x:1 if x>0 else 0)


In [37]:
# Pivoting France Data
france_pivot_data = pd.pivot_table(data=france_data,values= "Quantity",index="InvoiceNo",columns="Description",fill_value=0)
france_pivot_data = uk_pivot_data.applymap(lambda x:1 if x>0 else 0)

C:\Users\Chetan\AppData\Local\Temp\ipykernel_23240\1039042126.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  france_pivot_data = uk_pivot_data.applymap(lambda x:1 if x>0 else 0)


In [38]:
# Pivoting Germany Data
germany_pivot_data = pd.pivot_table(data=germany_data,values= "Quantity",index="InvoiceNo",columns="Description",fill_value=0)
germany_pivot_data = uk_pivot_data.applymap(lambda x:1 if x>0 else 0)

C:\Users\Chetan\AppData\Local\Temp\ipykernel_23240\1214522502.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  germany_pivot_data = uk_pivot_data.applymap(lambda x:1 if x>0 else 0)


### Applying Apriori Algorithm to find Frequent Bought together itemsets

In [70]:
uk_support = apriori(uk_pivot_data,min_support=0.03,use_colnames=True)

In [48]:
france_support = apriori(france_pivot_data,min_support=0.05,use_colnames=True)

In [54]:
germany_support = apriori(germany_pivot_data,min_support=0.05,use_colnames=True)

In [74]:
uk_support.shape,france_support.shape,germany_support.shape

((87, 2), (15, 2), (15, 2))

In [73]:
uk_data.shape

(494024, 7)

In [76]:
uk_data["Description"].nunique()

4202

### Apply Association Rules

In [89]:
association_rules(df=uk_support,min_threshold=0.25,metric="confidence",num_itemsets=4202)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.043421,0.042377,0.031806,0.732497,17.285056,1.0,0.029966,3.579862,0.984912,0.589076,0.720660,0.741516
1,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.042377,0.043421,0.031806,0.750535,17.285056,1.0,0.029966,3.834527,0.983839,0.589076,0.739212,0.741516
2,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.052586,0.087931,0.035617,0.677308,7.702719,1.0,0.030993,2.826438,0.918475,0.339533,0.646198,0.541182
3,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.087931,0.052586,0.035617,0.405057,7.702719,1.0,0.030993,1.592444,0.954068,0.339533,0.372034,0.541182
4,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.051270,0.087931,0.031670,0.617699,7.024813,1.0,0.027161,2.385736,0.903996,0.294515,0.580842,0.488932
5,(JUMBO BAG RED RETROSPOT),(JUMBO STORAGE BAG SUKI),0.087931,0.051270,0.031670,0.360165,7.024813,1.0,0.027161,1.482773,0.940332,0.294515,0.325588,0.488932


In [85]:
association_rules(df=germany_support,min_threshold=0.1,metric="confidence",num_itemsets=1703)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


In [88]:
association_rules(df=france_support,min_threshold=0.05,metric="confidence",num_itemsets=1565)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


In [86]:
france_data["Description"].nunique()

1565